In [1]:
#step one, get the top 100
import pandas as pd
from bs4 import BeautifulSoup
import requests

import os
# shazamapi needds ffmpeg , but it doesn't really have to be imported anywhere : import ffmpeg


In [13]:
#Initialize SpotiPy with user credentias
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

import combine_dataframe

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import pyplot
import numpy as np
import pickle

In [12]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 

In [4]:
all_song_dataframe, features_dataframe, credentials_dataframe = combine_dataframe.get_combinedSongs()

In [5]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,  client_secret= config.client_secret))
  

In [14]:
scaler = load("../models/scaler.pickle")

In [11]:
kmeans = load("../models/kmeans_135.pickle")



d:\installationen\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator KMeans from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [15]:
scaler.fit(features_dataframe)
audio_features_scaled = scaler.transform(features_dataframe)
category_dataframe = kmeans.predict(audio_features_scaled)

In [17]:
category_dataframe

array([ 62, 107, 115, ...,  52,  37, 108])

In [18]:
category_dataframe = pd.DataFrame(category_dataframe)
category_dataframe.columns = ['cluster']
search_data_dataframe = credentials_dataframe.join(category_dataframe)


In [24]:
# local libraries
import shazaam

In [31]:
def soundex(name):


    """
    The Soundex algorithm assigns a 1-letter + 3-digit code to strings,
    the intention being that strings pronounced the same but spelled
    differently have identical encodings; words pronounced similarly
    should have similar encodings.
    """

    soundexcoding = [' ', ' ', ' ', ' ']
    soundexcodingindex = 1

    #           ABCDEFGHIJKLMNOPQRSTUVWXYZ
    mappings = "01230120022455012623010202"

    soundexcoding[0] = name[0].upper()

    for i in range(1, len(name)):

         c = ord(name[i].upper()) - 65

         if c >= 0 and c <= 25:

             if mappings[c] != '0':

                 if mappings[c] != soundexcoding[soundexcodingindex-1]:

                     soundexcoding[soundexcodingindex] = mappings[c]
                     soundexcodingindex += 1

                 if soundexcodingindex > 3:

                     break

    if soundexcodingindex <= 3:
        while(soundexcodingindex <= 3):
            soundexcoding[soundexcodingindex] = '0'
            soundexcodingindex += 1

    return ''.join(soundexcoding)

In [ ]:
def is_recognizable():
    return len(shazam_info[1]["matches"])>0

def get_song_info():
    return shazam_info[1]["track"]["title"], shazam_info[1]["track"]["subtitle"]

In [28]:
URL = 'https://www.billboard.com/charts/hot-100/'
Top100Website = requests.get(URL)
result = BeautifulSoup(Top100Website.content, 'html.parser')

In [29]:
result.find("span", id="count_texttitle-of-a-story")

In [32]:
line_number = 0
place_number = 0
Song = "";
Artists = "";
Top_100 = pd.DataFrame(columns=['place','song','artists', 'song_soundex'])
print(str(Top_100))

for lines in result.find_all("li", {"o-chart-results-list__item"}):
    line_number += 1

 
    if((line_number-4)%14==0):
        place_number += 1
        internal_line = 1
        for current_line in lines.get_text().splitlines():

            if(current_line.strip()!=""):
                if(internal_line==1):
                    Song = current_line.strip()
                else:
                    Artists = current_line.strip()

                    Top_100.loc[place_number -1] = [place_number, Song, Artists, soundex(str(Song))]
 
                internal_line += 1

Empty DataFrame
Columns: [place, song, artists, song_soundex]
Index: []


In [33]:
#temporary
Top_100

,place,song,artists,song_soundex
0,1,As It Was,Harry Styles,A232
1,2,First Class,Jack Harlow,F623
2,3,Heat Waves,Glass Animals,H312
3,4,Big Energy,Latto,B256
4,5,Enemy,Imagine Dragons X JID,E500
...,...,...,...,...
95,96,P Power,Gunna Featuring Drake,P160
96,97,Money So Big,Yeat,M521
97,98,Blick Blick!,Coi Leray & Nicki Minaj,B421
98,99,Fall In Love,Bailey Zimmerman,F454


In [34]:
import config
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,  client_secret= config.client_secret))


In [35]:
recommend_me_something_for_this = input("Please enter a song to base a recommendation on ")

Please enter a song to base a recommendation on coca cola


In [36]:
song_selector_dictionary = {}
song_selected = ""
if(recommend_me_something_for_this=="shazaam"):
    #use the local shazaam library to identify the song currently playing
    recorded = shazaam.shazaam()

    #returns false if nothing was found
    if(recorded != False):
        #and a tuple of song and artist if it worked
        recorded_song, recorded_artist = recorded 
        print(recorded_song + " " + recorded_artist)
    
        #replace the input with the actual song
        recommend_me_something_for_this = recorded_song
  
Number_Of_Songs_Found = len(Top_100[Top_100['song_soundex'] == soundex(recommend_me_something_for_this.strip())])
if(Number_Of_Songs_Found>0):
    #find a new song
    print("Let me recommend you:")
    recommendation_dataframe = Top_100.sample(n=1)
    print(str(recommendation_dataframe ))
else:
    if(len(recommend_me_something_for_this.strip())>0):
        search_results = sp.search(q=recommend_me_something_for_this.strip(),limit=50,market="GB")
        if(len(search_results["tracks"])>1):
            print("Please select a number regarding the many titles of this name")
            for i in range(0,len(search_results["tracks"]["items"])):

                print("["+str(i+1)+"] " + str(search_results["tracks"]["items"][i]["name"]))
                print(str(search_results["tracks"]["items"][i]["uri"]))
                song_selector_dictionary[str(i+1)] = str(search_results["tracks"]["items"][i]["id"])
                print("by")
                for j in range(0,len(search_results["tracks"]["items"][i]["artists"])):

                    print(str(search_results["tracks"]["items"][i]["artists"][j]["name"]))
                print(" ")
           # print(str(search_results["tracks"]["items"][i]["name"]))        
        else:
            if(len(search_results)==1):
                print(str(search_results["tracks"]["items"][0]["name"]))
                print("by")
                for j in range(0,len(search_results["tracks"]["items"][0]["artists"])):
                    print("  " + str(search_results["tracks"]["items"][0]["artists"][j]["name"]))

        song_selected = input("which number do you choose? ")
    # print("“Unfortunately, the song is not in the hot list!")    

Please select a number regarding the many titles of this name
[1] Coca Cola (From "Luka Chuppi")
spotify:track:352wEkChkiClswqwDoM0nj
by
Tony Kakkar
Neha Kakkar
Young Desi
Tanishk Bagchi
 
[2] Colors - Coca-Cola® Anthem, 2018 FIFA World CupTM
spotify:track:7wD6iRghe6SnKHkvHqpnC5
by
Jason Derulo
 
[3] Coca Cola
spotify:track:7maTUpI64BKTkh6q7M1ZJD
by
FiloChill
 
[4] Wavin' Flag - Coca-Cola® Celebration Mix
spotify:track:6WFQsZZa81anp9BkqZMyTb
by
K'NAAN
 
[5] Cola Song (feat. J Balvin)
spotify:track:2ZAOEPkOeZTGsjGHWyCThc
by
INNA
J Balvin
 
[6] Coca Cola Sky
spotify:track:1lLEFznU4FGpxEFGg5TkkR
by
Kean Kavanagh
 
[7] Wavin' Flag - Coca-Cola® Celebration Mix
spotify:track:4NZvhN4bENiMBxlxyHujlZ
by
K'NAAN
 
[8] Cola
spotify:track:1HrMWH5GUdK6Yi94rbANJA
by
CamelPhat
Elderbrook
 
[9] Coca Cola Shape
spotify:track:5Q0QFhG1pkTZOFjwa3XNij
by
Simpleton
 
[10] Coca Cola (From "Luka Chuppi")
spotify:track:2hOlxSH3Zck6a2ki4xtKGV
by
Tony Kakkar
Neha Kakkar
Young Desi
 
[11] Rum And Coca-Cola
spotify

In [37]:
track_id = song_selector_dictionary[song_selected]
selection_features = sp.audio_features("spotify:track:" + track_id)
print(str(selection_features))
list_of_one = []
list_values_of_one = [
    selection_features[0]['danceability'],
    selection_features[0]['energy'],
    selection_features[0]['key'],
    selection_features[0]['loudness'],
    selection_features[0]['mode'],
    selection_features[0]['speechiness'],
    selection_features[0]['acousticness'],
    selection_features[0]['instrumentalness'],
    selection_features[0]['liveness'],
    selection_features[0]['valence'],
    selection_features[0]['tempo']
]
list_of_one.append(list_values_of_one)
dataframe_of_one = pd.DataFrame(list_of_one)
dataframe_of_one.columns = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']

scaler.fit(dataframe_of_one)
dataframe_of_one_scaled = scaler.transform(dataframe_of_one)
category_of_one_dataframe = kmeans.predict(dataframe_of_one_scaled)





[{'danceability': 0.707, 'energy': 0.845, 'key': 8, 'loudness': -3.478, 'mode': 1, 'speechiness': 0.201, 'acousticness': 0.0278, 'instrumentalness': 0, 'liveness': 0.134, 'valence': 0.507, 'tempo': 120.125, 'type': 'audio_features', 'id': '7wD6iRghe6SnKHkvHqpnC5', 'uri': 'spotify:track:7wD6iRghe6SnKHkvHqpnC5', 'track_href': 'https://api.spotify.com/v1/tracks/7wD6iRghe6SnKHkvHqpnC5', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7wD6iRghe6SnKHkvHqpnC5', 'duration_ms': 187390, 'time_signature': 4}]


In [38]:
catecory_to_find_song = category_of_one_dataframe[0]
track = search_data_dataframe.loc[search_data_dataframe['cluster'] == catecory_to_find_song ].sample(n=1)
print(str(track))
for r in track.to_numpy().tolist():
    track_id = r[1]


         Title                      id  cluster
350  Pardon Me  6RldLi8E0Q6nr8yfUnfXVQ       16


In [39]:
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

